In [2]:
# !pip3 install textgrid
import textgrid

In [3]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [4]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob
import soundfile as sf

os.makedirs('part6-1', exist_ok = True)
os.makedirs('part6-1-audio', exist_ok = True)
os.makedirs('part6-2', exist_ok = True)
os.makedirs('part6-2-audio', exist_ok = True)
os.makedirs('part6-3', exist_ok = True)
os.makedirs('part6-3-audio', exist_ok = True)

In [5]:
!mkdir part6-1-audio-mp3
!mkdir part6-2-audio-mp3
!mkdir part6-3-audio-mp3

mkdir: cannot create directory ‘part6-1-audio-mp3’: File exists
mkdir: cannot create directory ‘part6-2-audio-mp3’: File exists
mkdir: cannot create directory ‘part6-3-audio-mp3’: File exists


In [6]:
session = boto3.Session()

In [7]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [8]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 1/Scripts')):
    filename = os.path.join('part6-1', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

3978it [00:01, 2267.08it/s]


In [9]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 2/Scripts')):
    filename = os.path.join('part6-2', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

3966it [00:01, 3324.12it/s]


In [10]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 3/Scripts')):
    filename = os.path.join('part6-3', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

4038it [00:01, 3056.02it/s]


In [11]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 1/Audio')):
    filename = os.path.join('part6-1-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

3985it [00:01, 3514.14it/s]


In [12]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 2/Audio')):
    filename = os.path.join('part6-2-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

3966it [00:01, 3226.40it/s]


In [13]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART6/Call Centre Design 3/Audio')):
    filename = os.path.join('part6-3-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

4044it [00:01, 3109.71it/s]


In [23]:
import re

def cleaning(s):
    return re.sub(r'[ ]+', ' ', s).strip()

replaces = [
    '<malay>',
    '</malay>',
    '_',
    '<UNK>',
    '(ppl)',
    '(ppo)',
    '(mm)',
    '(um)',
    '(ppb)',
    '(mmhmm)',
    '<mandarin>',
    '</mandarin>'
]

replaces_with = {
    '[lah]': 'lah',
    '[ah]': 'ah',
    '[sia]': 'sia',
    '[eh]': 'eh',
    '(uh)': 'uh',
    '[what]': 'what',
    '[oh]': 'oh',
    '(err)': 'err',
    '[lor]': 'lor',
    '[ha]': 'ha',
    '[meh]': 'meh',
    '[one]': 'one',
    "[a'ah]": "a'ah",
    '[hor]': 'hor',
    '[leh]': 'leh',
    '[mah]': 'mah',
    '[nah]': 'nah',
    '[tau]': 'tau',
    '[uh]': 'uh',
    '[wah]': 'wah'
}

maxlen = 15

In [24]:
scripts = sorted(glob('part6-1/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((t, [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 3978/3978 [00:09<00:00, 413.70it/s]


In [25]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [28]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part6-1-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part6-1-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [29]:
import mp

results = mp.multiprocessing(keys, loop, cores = 20)

100%|██████████| 198/198 [37:36<00:00, 11.39s/it]


In [30]:
len(results)

117890

In [31]:
results[-2]

{'filename': 'part6-1-audio-mp3/app_0682_4364_phnd_cc-res-30.mp3',
 'text': 'okay, alright,'}

In [32]:
ipd.Audio(results[-2]['filename'], rate = 16000)

In [33]:
import json

with open('imda-part6-1.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 117890/117890 [00:00<00:00, 307288.67it/s]


In [34]:
!du -hs part6-1-audio-mp3

11G	part6-1-audio-mp3


In [35]:
scripts = sorted(glob('part6-2/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((', '.join(texts), [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 3966/3966 [00:16<00:00, 238.78it/s]


In [36]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [37]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part6-2-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part6-2-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [38]:
results_2 = mp.multiprocessing(keys, loop, cores = 20)

100%|██████████| 198/198 [35:16<00:00, 10.69s/it]


In [39]:
len(results_2)

108538

In [40]:
results_2[1000]

{'filename': 'part6-2-audio-mp3/app_0690_4380_phnd_cc-bnk-1.mp3',
 'text': "okay for me I actually lost a my credit card so I'm trying to find a replacement"}

In [41]:
ipd.Audio(results_2[1000]['filename'], rate = 16000)

In [43]:
with open('imda-part6-2.jsonl', 'w') as fopen:
    for r in tqdm(results_2):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 108538/108538 [00:00<00:00, 311224.22it/s]


In [44]:
scripts = sorted(glob('part6-3/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((', '.join(texts), [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 4038/4038 [00:12<00:00, 315.25it/s]


In [45]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [47]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part6-3-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part6-3-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [48]:
results_3 = mp.multiprocessing(keys, loop, cores = 20)

 81%|████████  | 162/201 [29:54<07:28, 11.49s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
len(results_3)

107820

In [51]:
results_3[1000]

{'filename': 'part6-3-audio-mp3/app_1356_0036_phnd_cc-hdb-4.mp3',
 'text': ', uh no that is uh but uh ya uh any bigger units will be resale flat,'}

In [52]:
ipd.Audio(results_3[1000]['filename'], rate = 16000)

In [53]:
with open('imda-part6-3.jsonl', 'w') as fopen:
    for r in tqdm(results_3):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 107820/107820 [00:00<00:00, 305758.01it/s]


In [54]:
from huggingface_hub import HfApi
api = HfApi()

In [56]:
for f in glob('imda-part6-*.jsonl'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

imda-part6-1.jsonl


imda-part6-1.jsonl:   0%|          | 0.00/26.9M [00:00<?, ?B/s]

imda-part6-3.jsonl


imda-part6-3.jsonl:   0%|          | 0.00/25.3M [00:00<?, ?B/s]

imda-part6-2.jsonl


imda-part6-2.jsonl:   0%|          | 0.00/24.9M [00:00<?, ?B/s]

In [58]:
# !sudo apt install p7zip-full p7zip-rar -y

In [65]:
for f in glob('part6-*-audio-mp3'):
    cmd = f'7z -v5g a {f}.7z {f} > /dev/null'
    print(f, cmd)
    os.system(cmd)

part6-2-audio-mp3 7z -v5g a part6-2-audio-mp3.7z part6-2-audio-mp3 > /dev/null
part6-3-audio-mp3 7z -v5g a part6-3-audio-mp3.7z part6-3-audio-mp3 > /dev/null
part6-1-audio-mp3 7z -v5g a part6-1-audio-mp3.7z part6-1-audio-mp3 > /dev/null


In [67]:
!du -hs part6-1-audio-mp3

11G	part6-1-audio-mp3


In [68]:
!du -hs part6-2-audio-mp3

10G	part6-2-audio-mp3


In [69]:
!du -hs part6-3-audio-mp3

10G	part6-3-audio-mp3


In [66]:
!ls *.7z*

part6-1-audio-mp3.7z.001  part6-2-audio-mp3.7z.001  part6-3-audio-mp3.7z.001
part6-1-audio-mp3.7z.002  part6-2-audio-mp3.7z.002  part6-3-audio-mp3.7z.002


In [70]:
for f in glob('*.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part6-3-audio-mp3.7z.001


part6-3-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part6-2-audio-mp3.7z.002


part6-2-audio-mp3.7z.002:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

part6-1-audio-mp3.7z.002


...

part6-1-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part6-3-audio-mp3.7z.002


part6-3-audio-mp3.7z.002:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

part6-2-audio-mp3.7z.001


part6-2-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [73]:
glob('part6-*-audio')

['part6-3-audio', 'part6-1-audio', 'part6-2-audio']

In [74]:
total_len = 0
for folder in glob('part6-*-audio'):
    folder = os.path.join(folder, '*.wav')
    print(folder)
    for f in tqdm(glob(folder)):
        y, sr = sf.read(f)
        total_len += len(y) / sr

part6-3-audio/*.wav


100%|██████████| 4044/4044 [02:11<00:00, 30.74it/s]


part6-1-audio/*.wav


100%|██████████| 3984/3984 [02:29<00:00, 26.72it/s]


part6-2-audio/*.wav


100%|██████████| 3966/3966 [02:19<00:00, 28.38it/s]


In [76]:
total_len / 60 / 60

2148.2834793402703

In [ ]:
!rm -rf part6-*-audio

In [78]:
!rm *7z*